In [2]:
import pandas as pd
from matplotlib import pyplot as plt
df = pd.read_csv('./Data/mitbih_train.csv')
df2 =pd.read_csv('./Data/mitbih_test.csv')

In [3]:

X = df.iloc[:, :-1]  # All columns except the last
Y = df.iloc[:, -1]   # Only the last column

# Check the shapes
print(f"Shape of X: {X.shape}")
print(f"Shape of Y: {Y.shape}")


Shape of X: (87553, 187)
Shape of Y: (87553,)


In [4]:
Xtest,Ytest = df2.iloc[:, :-1],df2.iloc[:, -1]

In [77]:
import torch
import torch.nn as nn

class myModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(myModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),  # First dense layer
            nn.ReLU(),                  # Activation function
            nn.Dropout(0.2),            # Dropout for regularization
            nn.Linear(128, 64),         # Second dense layer
            nn.ReLU(),                  # Activation function
            nn.Dropout(0.2),            # Dropout for regularization
            nn.Linear(64, num_classes)  # Output layer for classification
        )

    def forward(self, x):
        return self.model(x)


In [78]:

input_size = 187  # Number of features in X
num_classes = 5
model = myModel(input_size=input_size, num_classes=num_classes)


In [79]:
from torch.utils.data import DataLoader, TensorDataset

# Convert data to tensors
X_tensor = torch.tensor(X.values, dtype=torch.float32)  # Convert X to tensor
Y_tensor = torch.tensor(Y.values, dtype=torch.long)     # Convert Y to tensor

# Create DataLoader for batching
batch_size = 64
dataset = TensorDataset(X_tensor, Y_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [80]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()  # Loss function for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [81]:
epochs = 50

# Training loop
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for batch_idx, (inputs, labels) in enumerate(dataloader):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

    # Print epoch statistics
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

print("Training complete!")

Epoch [1/50], Loss: 0.3489
Epoch [2/50], Loss: 0.1967
Epoch [3/50], Loss: 0.1647
Epoch [4/50], Loss: 0.1473
Epoch [5/50], Loss: 0.1355
Epoch [6/50], Loss: 0.1278
Epoch [7/50], Loss: 0.1214
Epoch [8/50], Loss: 0.1164
Epoch [9/50], Loss: 0.1120
Epoch [10/50], Loss: 0.1076
Epoch [11/50], Loss: 0.1038
Epoch [12/50], Loss: 0.1036
Epoch [13/50], Loss: 0.1017
Epoch [14/50], Loss: 0.0966
Epoch [15/50], Loss: 0.0961
Epoch [16/50], Loss: 0.0961
Epoch [17/50], Loss: 0.0936
Epoch [18/50], Loss: 0.0920
Epoch [19/50], Loss: 0.0917
Epoch [20/50], Loss: 0.0878
Epoch [21/50], Loss: 0.0894
Epoch [22/50], Loss: 0.0857
Epoch [23/50], Loss: 0.0854
Epoch [24/50], Loss: 0.0839
Epoch [25/50], Loss: 0.0834
Epoch [26/50], Loss: 0.0828
Epoch [27/50], Loss: 0.0807
Epoch [28/50], Loss: 0.0811
Epoch [29/50], Loss: 0.0801
Epoch [30/50], Loss: 0.0775
Epoch [31/50], Loss: 0.0787
Epoch [32/50], Loss: 0.0774
Epoch [33/50], Loss: 0.0790
Epoch [34/50], Loss: 0.0770
Epoch [35/50], Loss: 0.0756
Epoch [36/50], Loss: 0.0775
E

In [82]:
# Convert test data to tensors
Xtest_tensor = torch.tensor(Xtest.values, dtype=torch.float32)
Ytest_tensor = torch.tensor(Ytest.values, dtype=torch.long)
print(f"Shape of X: {Xtest_tensor.shape}")
print(f"Shape of Y: {Ytest_tensor.shape}")

Shape of X: torch.Size([21891, 187])
Shape of Y: torch.Size([21891])


In [83]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient computation
    # Forward pass for all test samples
    outputs = model(Xtest_tensor)  # Shape: (num_samples, num_classes)
    _, predicted = torch.max(outputs, 1)  # Get the predicted class for each sample

    # Calculate accuracy
    correct = (predicted == Ytest_tensor).sum().item()
    total = Ytest_tensor.size(0)
    accuracy = correct / total * 100

print(f"Accuracy on test set: {accuracy:.2f}%")

Accuracy on test set: 97.88%


In [84]:
torch.save(model.state_dict(), "myModel.pth")


In [85]:
import torch.quantization as quant

# Set model to evaluation mode
model.eval()

# Apply dynamic quantization
quantized_model = quant.quantize_dynamic(
    model, 
    {torch.nn.Linear},  # Specify the layers to quantize
    dtype=torch.qint8   # Use 8-bit integers for quantization
)

# Evaluate the quantized model
with torch.no_grad():
    outputs = quantized_model(Xtest_tensor)
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == Ytest_tensor).sum().item()
    total = Ytest_tensor.size(0)
    quantized_accuracy = correct / total * 100

print(f"Accuracy on test set (quantized model): {quantized_accuracy:.2f}%")


Accuracy on test set (quantized model): 97.88%


In [86]:
print(quantized_model.state_dict())

OrderedDict([('model.0.scale', tensor(1.)), ('model.0.zero_point', tensor(0)), ('model.0._packed_params.dtype', torch.qint8), ('model.0._packed_params._packed_params', (tensor([[-0.0445, -0.0445, -0.0445,  0.0000, -0.0891,  0.0000,  0.0000, -0.0891,
          0.0000, -0.0891,  0.0000, -0.0445,  0.0445, -0.0891,  0.0445, -0.0445,
          0.0000,  0.0000,  0.0445,  0.0445, -0.0445, -0.0445, -0.0445,  0.0445,
         -0.0445,  0.0445, -0.0445,  0.0000, -0.0891, -0.0445, -0.0445,  0.0445,
         -0.0445, -0.0891, -0.0891,  0.0000, -0.0445, -0.0891,  0.0000, -0.0445,
          0.0000,  0.0000, -0.0891,  0.0000,  0.0445, -0.0891,  0.0000,  0.0000,
          0.0000, -0.1336,  0.0445,  0.0445,  0.0000, -0.0445, -0.0891, -0.0891,
          0.0445, -0.0445, -0.0445,  0.0000,  0.0000, -0.0445,  0.0000, -0.0891,
         -0.0445, -0.0891,  0.0000,  0.0000, -0.0445, -0.0445,  0.0445, -0.0445,
         -0.0445,  0.0000,  0.0445, -0.0445,  0.0000,  0.0445, -0.0445,  0.0445,
         -0.0445,  0.

In [89]:
# torch.set_printoptions(threshold=100000000)  # Or a higher number if needed
# file_path = 'ordered_dict.txt'
# with open(file_path, 'w') as file:
#     for key, value in quantized_model.state_dict().items():
#         file.write(f"{key}: {value}\n")

torch.save(quantized_model,"quantized.pth")